In [2]:
%%capture
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import binary_fill_holes
from skimage.measure import regionprops
from skimage.measure import label

In [3]:
# HYPERPARAMETERS
IMAGE_SIZE = [5184 // 10, 3456 // 10]
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 20

In [4]:
from keras.utils import image_dataset_from_directory
from keras.utils import to_categorical

num_classes = 6

train_ds = image_dataset_from_directory("Dataset-preprocessed/Train",
                                        validation_split=0.2,
                                        image_size=IMAGE_SIZE,
                                        seed=42,
                                        subset="training",
                                        label_mode='categorical')

val_ds = image_dataset_from_directory("Dataset-preprocessed/Train",
                                        validation_split=0.2,
                                        image_size=IMAGE_SIZE,
                                        seed=42,
                                        subset="validation",
                                        label_mode='categorical')

test_ds = image_dataset_from_directory("Dataset-preprocessed/Test",image_size=IMAGE_SIZE,
                                        label_mode='categorical')


Found 1612 files belonging to 6 classes.
Using 1290 files for training.


2024-08-26 19:21:05.404844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-26 19:21:05.423537: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-26 19:21:05.423599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-26 19:21:05.426837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-26 19:21:05.426950: I tensorflow/compile

Found 1612 files belonging to 6 classes.
Using 322 files for validation.
Found 180 files belonging to 6 classes.


In [5]:
from keras import Sequential
from keras.layers import Conv2D, Flatten, Dense, Input
from keras.layers import Dropout, BatchNormalization, MaxPooling2D
from keras.layers import RandomFlip, RandomRotation, Rescaling
from keras.metrics import sparse_categorical_crossentropy
from keras.optimizers import Adam
from keras.applications.resnet_v2 import ResNet50V2

num_classes = 6

model = Sequential()

model.add(Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
model.add(Rescaling(1./255))
model.add(RandomFlip("horizontal_and_vertical", seed=42))
model.add(RandomRotation(1, seed=42))

model.add(Conv2D(32, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(32, (3, 3), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), activation='relu', padding="same"))
model.add(Conv2D(64, (5, 5), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (5, 5), activation='relu', padding="same"))
model.add(Conv2D(128, (5, 5), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (5, 5), activation='relu', padding="same"))
model.add(Conv2D(256, (5, 5), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(
  optimizer=Adam(learning_rate=LEARNING_RATE),
  loss='categorical_crossentropy',
  metrics=['accuracy'])

model.summary()
# base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 518, 345, 3)       0         
                                                                 
 random_flip (RandomFlip)    (None, 518, 345, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 518, 345, 3)      0         
 tion)                                                           
                                                                 
 conv2d (Conv2D)             (None, 518, 345, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 518, 345, 32)      9248      
                                                                 
 batch_normalization (BatchN  (None, 518, 345, 32)     128       
 ormalization)                                          

In [6]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

my_callbacks = [
    # EarlyStopping(patience=5),
    ModelCheckpoint(filepath='models/model.{epoch:02d}.keras', save_best_only=True)
]


H = model.fit(
    train_ds,
    validation_data=val_ds,  # Validation data for evaluation during training
    epochs=EPOCHS,  # Number of epochs to train the model
    callbacks=my_callbacks) # Number of batches in validation)

Epoch 1/100


2024-08-26 19:21:41.030218: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1290]
	 [[{{node Placeholder/_0}}]]
2024-08-26 19:21:41.030487: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [1290]
	 [[{{node Placeholder/_4}}]]
2024-08-26 19:21:41.687916: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-08-26 19:21:42.144098: I tensorflow/compil

41/41 [==============================] - ETA: 0s - loss: 12.7435 - accuracy: 0.2814

2024-08-26 19:22:47.122984: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [322]
	 [[{{node Placeholder/_4}}]]
2024-08-26 19:22:47.123241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [322]
	 [[{{node Placeholder/_0}}]]


41/41 [==============================] - 69s 901ms/step - loss: 12.7435 - accuracy: 0.2814 - val_loss: 34.6492 - val_accuracy: 0.1832
Epoch 2/100
41/41 [==============================] - 26s 628ms/step - loss: 1.5735 - accuracy: 0.3395 - val_loss: 23.2215 - val_accuracy: 0.1832
Epoch 3/100
41/41 [==============================] - 26s 628ms/step - loss: 1.5306 - accuracy: 0.3690 - val_loss: 19.8247 - val_accuracy: 0.1832
Epoch 4/100
41/41 [==============================] - 26s 633ms/step - loss: 1.5198 - accuracy: 0.3636 - val_loss: 18.4613 - val_accuracy: 0.1832
Epoch 5/100
41/41 [==============================] - 26s 624ms/step - loss: 1.5380 - accuracy: 0.3341 - val_loss: 18.8079 - val_accuracy: 0.1832
Epoch 6/100
41/41 [==============================] - 26s 621ms/step - loss: 1.4878 - accuracy: 0.3659 - val_loss: 19.4525 - val_accuracy: 0.1832
Epoch 7/100
41/41 [==============================] - 26s 622ms/step - loss: 1.4953 - accuracy: 0.3581 - val_loss: 19.2294 - val_accuracy: 0.1

KeyboardInterrupt: 